In [1]:
import pandas as pd
import numpy as np

from pathlib import Path

DETECTORS = ["yolov5m", "yolov5x", "yolov7", "yolov7x", "yolor-csp"]
TRACKERS = ["deepsort", "fastreid"]

RESULTS_PATH = Path("../eval_results/MOT17")
RESULTS_FILENAME = "pedestrian_summary.txt"
DETAILS_FILENAME = "pedestrian_detailed.csv"
SPEED_FILENAME = "speed.txt"

dfs = {}
speeds = {}

In [2]:
for detector in DETECTORS:
    dfs[detector] = pd.read_csv(RESULTS_PATH.joinpath(detector,RESULTS_FILENAME), sep=" ")

In [3]:
for detector in DETECTORS:
    speeds[detector] = pd.read_csv(RESULTS_PATH.joinpath(detector,SPEED_FILENAME), sep=" ", header=None)
    speeds[detector].columns = ['Seq', 'DetSpeed', 'TrackSpeed']
    speeds[detector]['TotalSpeed'] = speeds[detector]['DetSpeed'] + speeds[detector]['TrackSpeed']
    speeds[detector]['FPS'] = (1000 / speeds[detector]['TotalSpeed']).round(1)
    #TODO: Chnage COMBINED logic
    # speeds[detector].loc[speeds[detector].index[-1]+1] = speeds[detector].mean(numeric_only=True)
    # speeds[detector].loc[speeds[detector].index[-1], 'Seq'] = 'COMBINED'
    speeds[detector].loc['COMBINED'] = speeds[detector].mean(numeric_only=True).round(1)
    speeds[detector].loc['COMBINED', 'Seq'] = 'COMBINED'

In [4]:
df = pd.DataFrame([],columns=list(['Detector', 'DetSpeed', 'TrackSpeed', 'TotalSpeed', 'FPS'] + (list(dfs['yolov7'].columns))))
for detector in DETECTORS:
    new_row = pd.Series(
        np.append(
            np.append([detector], speeds[detector].loc['COMBINED'][1:].values), 
            dfs[detector].values
            )
        ).to_frame().T
    new_row.columns = df.columns
    df = pd.concat([df, new_row], ignore_index=True)

In [5]:
df

,Detector,DetSpeed,TrackSpeed,TotalSpeed,FPS,HOTA,DetA,AssA,DetRe,DetPr,...,IDP,IDTP,IDFN,IDFP,Dets,GT_Dets,IDs,GT_IDs,SFDA,ATA
0,yolov5m,28.1,29.0,57.1,18.3,38.045,38.013,38.555,40.53,73.997,...,64.59,119184.0,217707.0,65340.0,184524.0,336891.0,852.0,1638.0,51.515,24.35
1,yolov5x,52.7,31.2,83.9,12.2,39.516,39.886,39.583,42.56,74.64,...,66.08,126939.0,209952.0,65160.0,192099.0,336891.0,942.0,1638.0,53.129,25.166
2,yolov7,41.2,28.1,69.4,14.8,39.115,37.927,40.656,39.9,77.507,...,67.784,117558.0,219333.0,55872.0,173430.0,336891.0,825.0,1638.0,51.654,23.814
3,yolov7x,53.5,28.8,82.3,12.4,39.483,38.734,40.707,40.772,77.396,...,69.948,124140.0,212751.0,53334.0,177474.0,336891.0,843.0,1638.0,52.383,25.667
4,yolor-csp,44.3,19.2,63.5,15.9,31.451,26.863,37.152,27.707,80.047,...,68.767,80187.0,256704.0,36420.0,116607.0,336891.0,567.0,1638.0,40.888,18.102
